In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import gzip
import numpy as np
from scipy.optimize import minimize
from datetime import timedelta
import tensorflow as tf

In [3]:
with gzip.open("/home/sathappan/user_timeseries_sorted.gz" ) as inf:
    cnt = 0
    d2 = []
    import json
    from dateutil.parser import parse
    for i in inf:
        d2.append([parse(dt) for dt in json.loads(i)['arrivalTimes']])
        cnt += 1
        if cnt > 100:
            break

In [4]:
from datetime import timedelta

In [5]:
dframe = pd.DataFrame(d2)
#dframe.fillna(timedelta(0))
df = dframe.loc[:, :5].copy()

In [6]:
dframe.columns = [str(l) for l in range(0, 1540)]
names = [str(l) for l in range(0, 1540)]

In [7]:
#df['newcol'] = df[['col1','col2']].sum(axis=1) - df['col5']
#df[['newcol', 'col3']].sub(df['col5'], axis=0)
arrivals = dframe[names[:400]].sub(dframe["0"], axis=0)
mask = ~(arrivals.isnull()).as_matrix()
arrivals_numpy = arrivals.applymap(lambda x: x/np.timedelta64(1, '1D')).as_matrix()

In [8]:
arrivals_numpy.shape

(101, 400)

In [9]:
#arrivals = df - df.loc[:, 0][:, None]
#arrivals.fillna(timedelta(0), inplace=True)
#arrivals = arrivals.loc[:, 1:].applymap(lambda x: x.to_timedelta64())

In [10]:
arrivals.fillna(timedelta(0), inplace=True)

In [27]:
def loglikelihood():
    #tdiff = diff(arrivals) * mask[:, 1:]
    #A_i = tf.zeros_like(arrivals)
    #for i in range(tdiff.shape[0]):
    #    A_i[:, i + 1] = tdiff[:, i] * (1 + A[:, i ])
    part1 = tf.reduce_sum(tf.log(mu + alpha * ais_tvar), axis=1)
        
    part2 = mu * T_max
    part3 = (alpha / beta)* tf.reduce_sum(tf.exp(-beta * ((tf.constant(725.0, dtype=tf.float64) - arrivals_tvar)*mask_tvar)) - tf.constant(1.0, dtype=tf.float64))
    
    return part1 - part2 + part3

In [12]:
def calc_Ai(arrivals, mask, beta):
    tdiff = np.diff(arrivals) * mask[:, 1:]
    tdiff = np.exp(tdiff * -beta)
    A_i = np.zeros_like(arrivals, dtype=np.float128)
    for i in range(tdiff.shape[1]):
        #print(i,)
        A_i[:, i + 1] = tdiff[:, i] * (1 + A_i[:, i])
    return A_i, tdiff
    

In [13]:
A_i, tempdiff = calc_Ai(np.nan_to_num(arrivals_numpy), mask, -1.0)

In [14]:
arrivals_numpy.shape

(101, 400)

In [17]:
np.nan_to_num(arrivals_numpy).max()

724.23681712962969

#### Tensorflow fitting

In [25]:
## Assuming number of users is 32
num_users = 101

## All parameters are column vectors with size equal to number of users

alpha = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), dtype=tf.float64)
beta = tf.Variable(tf.ones([num_users, 1], dtype=tf.float64), dtype=tf.float64)
mu = tf.Variable(tf.zeros([num_users, 1], dtype=tf.float64), dtype=tf.float64)


In [28]:
T_max = 400
arrivals_tvar = tf.placeholder(tf.float64, [None, T_max])
mask_tvar = tf.placeholder(tf.float64, [None, T_max])
ais_tvar = tf.placeholder(tf.float64,[None, T_max])
tdiff_tvar = tf.placeholder(tf.float64, [None, T_max-1])
loglik = loglikelihood()

In [30]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loglik, feed_dict={arrivals_tvar: np.nan_to_num(arrivals_numpy), mask_tvar:mask, ais_tvar:A_i, tdiff_tvar: tempdiff}))

[[-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 ..., 
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]
 [-inf -inf -inf ..., -inf -inf -inf]]
